# Rossman Store Sales Prediction

https://www.kaggle.com/c/rossmann-store-sales/overview

## Description
Rossmann operates over 3,000 drug stores in 7 European countries. Currently, Rossmann store managers are tasked with predicting their daily sales for up to six weeks in advance. Store sales are influenced by many factors, including promotions, competition, school and state holidays, seasonality, and locality. With thousands of individual managers predicting sales based on their unique circumstances, the accuracy of results can be quite varied.

In their first Kaggle competition, Rossmann is challenging you to predict 6 weeks of daily sales for 1,115 stores located across Germany. Reliable sales forecasts enable store managers to create effective staff schedules that increase productivity and motivation. By helping Rossmann create a robust prediction model, you will help store managers stay focused on what’s most important to them: their customers and their teams!

## Problem Statement
You are provided with historical sales data for 1,115 Rossmann stores. The task is to forecast the "Sales" column for the test set. Note that some stores in the dataset were temporarily closed for refurbishment.

## Evaluation
RMSPE
<br>
www.kaggle.com/competitions/rossmann-store-sales/overview/evaluation

## Data Dictionary
**Files**
* `train.csv` - historical data including Sales
* `test.csv` - historical data excluding Sales
* `sample_submission.csv` - a sample submission file in the correct format
* `store.csv` - supplemental information about the stores

**Data Fields**
<br>
Most of the fields are self-explanatory. The following are descriptions for those that aren't.

* `Id` - an Id that represents a (Store, Date) duple within the test set
* `Store` - a unique Id for each store
* `Sales` - the turnover for any given day (this is what you are predicting)
* `Customers` - the number of customers on a given day
* `Open` - an indicator for whether the store was open: 0 = closed, 1 = open
* `StateHoliday` - indicates a state holiday. Normally all stores, with few exceptions, are closed on state holidays. Note that all schools are closed on public holidays and weekends. a = public holiday, b = Easter holiday, c = Christmas, 0 = None
* `SchoolHoliday` - indicates if the (Store, Date) was affected by the closure of public schools
* `StoreType` - differentiates between 4 different store models: a, b, c, d
* `Assortment` - describes an assortment level: a = basic, b = extra, c = extended
* `CompetitionDistance` - distance in meters to the nearest competitor store
* `CompetitionOpenSince[Month/Year]` - gives the approximate year and month of the time the nearest competitor was opened
* `Promo` - indicates whether a store is running a promo on that day
* `Promo2` - Promo2 is a continuing and consecutive promotion for some stores: 0 = store is not participating, 1 = store is participating
* `Promo2Since[Year/Week]` - describes the year and calendar week when the store started participating in Promo2
* `PromoInterval` - describes the consecutive intervals Promo2 is started, naming the months the promotion is started anew. E.g. "Feb,May,Aug,Nov" means each round starts in February, May, August, November of any given year for that store

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sb

In [2]:
train = pd.read_csv("data/rossmann-store-sales/train.csv")
test = pd.read_csv("data/rossmann-store-sales/test.csv")
store = pd.read_csv("data/rossmann-store-sales/store.csv")

C:\Users\vanme\AppData\Local\Temp\ipykernel_9184\1315802104.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv("data/rossmann-store-sales/train.csv")


In [3]:
print(f"Training Data Shape: {train.shape}")
print(f"Test Data Shape: {test.shape}")
print(f"Store Data Shape: {store.shape}")

Training Data Shape: (1017209, 9)
Test Data Shape: (41088, 8)
Store Data Shape: (1115, 10)


In [4]:
train.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


In [5]:
test.head()

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday
0,1,1,4,2015-09-17,1.0,1,0,0
1,2,3,4,2015-09-17,1.0,1,0,0
2,3,7,4,2015-09-17,1.0,1,0,0
3,4,8,4,2015-09-17,1.0,1,0,0
4,5,9,4,2015-09-17,1.0,1,0,0


In [6]:
store.head()

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN


### Data Preprocessing

Checking if all the stores are open for which the sales values are greater than zero. Otherwise, remove those records.

In [10]:
not_open = train[(train['Open'] == 0) & (train['Sales'] != 0)]
print(f"No closed stores with sales: {str(not_open.size == 0)}")

no_sales = train[(train['Open'] == 1) & (train['Sales'] <= 0)]
print(f"No open stores with no sales: {str(no_sales.size == 0)}")

No closed stores with sales: True
No open stores with no sales: False


Removing records where the `Sales` column is less than or equal to zero.

In [11]:
train = train.loc[train['Sales'] > 0]

In [12]:
print(f"Training Data Shape After Cleaning: {train.shape}")

Training Data Shape After Cleaning: (844338, 9)


In [14]:
train.isnull().sum()

Store            0
DayOfWeek        0
Date             0
Sales            0
Customers        0
Open             0
Promo            0
StateHoliday     0
SchoolHoliday    0
dtype: int64

In [15]:
test.isnull().sum()

Id                0
Store             0
DayOfWeek         0
Date              0
Open             11
Promo             0
StateHoliday      0
SchoolHoliday     0
dtype: int64

In [16]:
store.isnull().sum()

Store                          0
StoreType                      0
Assortment                     0
CompetitionDistance            3
CompetitionOpenSinceMonth    354
CompetitionOpenSinceYear     354
Promo2                         0
Promo2SinceWeek              544
Promo2SinceYear              544
PromoInterval                544
dtype: int64